In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw

from src.input_pipeline import Pipeline

# Get images and boxes

In [ ]:
tf.reset_default_graph()

pipeline = Pipeline(
    ['data/val_shards/shard-0000.tfrecords'],
    batch_size=24, image_size=(512, 512),
    repeat=False, shuffle=False, 
    augmentation=True
)
features, labels = pipeline.get_batch()

In [ ]:
with tf.Session() as sess:
    sess.run(pipeline.init)
    I, B, L, N = sess.run([
        features['images'], labels['boxes'],
        labels['labels'], labels['num_boxes']
    ])

# Show an augmented image with boxes

In [ ]:
def draw_boxes(image, boxes, labels):
    image_copy = image.copy()
    draw = ImageDraw.Draw(image_copy, 'RGBA')
    width, height = image.size

    for box, label in zip(boxes, labels):
        ymin, xmin, ymax, xmax = box
        xmin, xmax = width*xmin, width*xmax
        ymin, ymax = height*ymin, height*ymax

        text = str(label)
        fill = (255, 255, 255, 45)
        outline = 'black'

        draw.rectangle(
            [(xmin, ymin), (xmax, ymax)],
            fill=fill, outline=outline
        )
        draw.rectangle(
            [(xmin, ymin), (xmin + 6*len(text) + 4, ymin + 12)],
            fill='white', outline='white'
        )
        draw.text((xmin + 2, ymin + 2), text, fill='red')

    return image_copy

In [ ]:
# choose an image
i = 0
image = Image.fromarray((np.transpose(I[i], [1, 2, 0])*255.0).astype('uint8'))
num_boxes = N[i]
boxes = B[i][:num_boxes]
classes = L[i][:num_boxes]

draw_boxes(image, boxes, classes)

# Test correctness of the pipeline

In [ ]:
# make sure that all images are read at least once

In [ ]:
tf.reset_default_graph()

filenames = [
    'data/val_shards/shard-0000.tfrecords', 
    'data/val_shards/shard-0001.tfrecords',
    'data/val_shards/shard-0002.tfrecords'
]
pipeline = Pipeline(
    filenames,
    batch_size=1, image_size=(512, 512),
    repeat=False, shuffle=True, 
    augmentation=False
)
features, labels = pipeline.get_batch()

In [ ]:
# test 1
read_filenames = []
with tf.Session() as sess:
    sess.run(pipeline.init)
    try:
        while True:
            F = sess.run(features['filenames'])
            read_filenames.append(F)
    except tf.errors.OutOfRangeError:
        pass
    
read_filenames = np.concatenate(read_filenames)

In [ ]:
# test 2
# set repeat=True
read_filenames = []
num_batches = 0
with tf.Session() as sess:
    sess.run(pipeline.init)
    while num_batches < 20:
        F = sess.run(features['filenames'])
        read_filenames.append(F)
        num_batches += 1

read_filenames = np.concatenate(read_filenames)

In [ ]:
print('number of images read:', len(read_filenames))
print('number of unique images read:', len(np.unique(read_filenames)))